In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text
import os

C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_4920\1853401532.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Load the data

In [2]:
train_files = [
    "validation/2024-05-13/2024-05-13_12.json",

]

test_files = [
    "validation/2024-05-13/treino-2024-05-13_13.json",
]

answer_files = [
    "validation/2024-05-13/resposta-2024-05-13_13.json"
]
train = pd.read_json(train_files[0], encoding='latin-1')
test = pd.read_json(test_files[0], encoding='latin-1')

## Filter data

In [3]:
train['latitude'] = train['latitude'].str.replace(',', '.').astype(float)
train['longitude'] = train['longitude'].str.replace(',', '.').astype(float)
train['linha'] = train['linha'].astype(str)

valid_linhas = [
    '483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', 
    '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', 
    '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', 
    '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', 
    '852', '557', '759', '343', '779', '905', '108'
]

df_train = train[train['linha'].isin(valid_linhas)]

## Create dataframe with last 2 points of each line and bus order

In [4]:
df_last_two = df_train.groupby(['ordem', 'linha']).tail(2).reset_index(drop=True)

counts = df_last_two.groupby(['ordem', 'linha']).size()
to_duplicate = counts[counts == 1].index

duplicated_rows = df_last_two.set_index(['ordem', 'linha']).loc[to_duplicate].reset_index()
df_last_two = pd.concat([df_last_two, duplicated_rows]).sort_values(['ordem', 'linha'])


df_last_two = df_last_two[['ordem','linha','latitude','longitude','datahoraservidor']]

## Join data to predict with last two points dataframe


In [5]:
test['linha'] = test['linha'].astype(str)

df_test = test[test['linha'].isin(valid_linhas)]

# Join the two dataframes
join_df = pd.merge(df_test, df_last_two, on=['ordem','linha'], how='inner')
join_df

,id,ordem,linha,datahora,latitude,longitude,datahoraservidor
0,1,B51503,917,1715612396000,-22.88219,-43.32685,1715612380000
1,1,B51503,917,1715612396000,-22.88206,-43.32547,1715612411000
2,2,B51558,624,1715612395000,-22.89998,-43.22958,1715612380000
3,2,B51558,624,1715612395000,-22.90037,-43.23099,1715612411000
4,3,B51573,917,1715612393000,-22.88306,-43.29619,1715612380000
...,...,...,...,...,...,...,...
310871,156640,B31083,483,1715615993000,-22.89362,-43.21548,1715612394000
310872,156641,B31089,483,1715615994000,-22.91135,-43.16892,1715612364000
310873,156641,B31089,483,1715615994000,-22.90865,-43.16830,1715612394000
310874,156642,B31130,483,1715615995000,-22.92101,-43.17155,1715612364000


## Model - Predict the next position of the bus based on past positions

In [6]:
database_url = os.getenv("DATABASE_URL")
engine = create_engine(database_url, client_encoding='latin-1')  

In [7]:
def execute_query(connection, linha, lat1, lon1, lat2, lon2, last_date, prediction_date):
    query = """
    WITH initial_similar_points AS (
        SELECT time_ranking,
               ordem,
               linha,
               x,
               y,
               datahoraservidor
        FROM vw_buses_order
        WHERE linha = :linha
        AND x = width_bucket(:lon1, -43.726090, -42.951470, 1587)
        AND y = width_bucket(:lat1, -23.170790, -22.546410, 1389)
        AND (
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '7 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '7 day' + interval '2 hour') 
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '14 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '14 day' + interval '2 hour')
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '21 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '21 day' + interval '2 hour')
            )
        AND time_ranking > 1
        LIMIT 10
    ), anterior_points AS (
        SELECT DISTINCT ON (time_ranking, ordem, linha) 
            time_ranking,
            ordem,
            linha,
            x,
            y,
            datahoraservidor
        FROM vw_buses_order
        WHERE (ordem, linha, time_ranking) IN (
            SELECT ordem, linha, time_ranking - 1
            FROM initial_similar_points
            )
    ), direction_points AS (
         SELECT 
            sp.ordem,
            sp.datahoraservidor
        FROM initial_similar_points sp
        INNER JOIN anterior_points ap
            ON sp.ordem = ap.ordem
            AND sp.linha = ap.linha
            AND sp.time_ranking = ap.time_ranking + 1
        WHERE ((ap.x - sp.x) * (:lon2 - :lon1) + (ap.y - sp.y) * (:lat2 - :lat1)) >= 0
    ), first_future_points AS (
        SELECT DISTINCT ON (vo.ordem, vo.datahoraservidor)
            vo.x,
            vo.y,
            vo.ordem,
            vo.datahoraservidor              
        FROM (
                SELECT 
                          ordem,
                          linha,
                          x,
                          y,
                          datahoraservidor
                FROM vw_buses_order
                WHERE linha = :linha
                AND ordem IN (SELECT DISTINCT ordem FROM direction_points)
             ) vo
        INNER JOIN direction_points dp
            ON vo.ordem = dp.ordem
            AND vo.datahoraservidor > dp.datahoraservidor
            AND vo.datahoraservidor < dp.datahoraservidor + interval '1 hour'
        WHERE vo.datahoraservidor > dp.datahoraservidor + (TO_TIMESTAMP(:prediction_date) - TO_TIMESTAMP(:last_date) - interval '2 minutes')
        AND vo.datahoraservidor < dp.datahoraservidor + (TO_TIMESTAMP(:prediction_date) - TO_TIMESTAMP(:last_date) + interval '2 minutes')
    ), selected_future_points AS (
        SELECT x,y
        FROM first_future_points
    )
    SELECT 
        ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY x)) AS median_x,
        ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY y)) AS median_y
    FROM selected_future_points;
    """
    
    params = {
        'linha': linha,
        'lat1': lat1,
        'lon1': lon1,
        'lat2': lat2,
        'lon2': lon2,
        'last_date': str(last_date),
        'prediction_date': str(prediction_date)
    }
    
    
    result = connection.execute(text(query), params)
    row = result.fetchone()
        
    return row[0], row[1]

In [8]:
median_x_list = []
median_y_list = []

with engine.connect() as connection:
    for i in range(0, len(join_df)- 1, 2):
        row1 = join_df.iloc[i + 1]
        row2 = join_df.iloc[i]
        median_x, median_y = execute_query(
            connection,
            row1['linha'], 
            row1['latitude'], 
            row1['longitude'], 
            row2['latitude'], 
            row2['longitude'], 
            row1['datahoraservidor']/1000, # Convert to seconds - Last Date
            row1['datahora']/1000 # Convert to seconds - Prediction Date
        )

        median_x_list.extend([median_x, median_x])
        median_y_list.extend([median_y, median_y])

join_df['median_x'] = median_x_list
join_df['median_y'] = median_y_list

df_prediction = join_df[['id','latitude', 'longitude', 'median_y','median_x']]

### Count nulls (%)

In [ ]:
(df_prediction.isnull().sum()/len(df_prediction)) * 100

id                  0.000000
ordem               0.000000
linha               0.000000
datahora            0.000000
latitude            0.000000
longitude           0.000000
datahoraservidor    0.000000
median_x            3.280408
median_y            3.280408
dtype: float64

### Convert the prediction to latitude and longitude

In [ ]:
def inverse_width_bucket_x(bucket_index):
    min_value = -43.726090
    max_value = -42.951470
    num_buckets = 1587

    if bucket_index < 1:
        return min_value
    elif bucket_index > num_buckets:
        return max_value
    
    bucket_size = (max_value - min_value) / num_buckets
    value = min_value + (bucket_index - 1) * bucket_size + bucket_size / 2
    
    return value
    
def inverse_width_bucket_y(bucket_index):
    min_value = -23.170790
    max_value = -22.546410
    num_buckets = 1389

    if bucket_index < 1:
        return min_value
    elif bucket_index > num_buckets:
        return max_value
    
    bucket_size = (max_value - min_value) / num_buckets
    value = min_value + (bucket_index - 1) * bucket_size + bucket_size / 2
    
    return value

df_prediction['median_x'] = df_prediction['median_x'].apply(inverse_width_bucket_x)
df_prediction['median_y'] = df_prediction['median_y'].apply(inverse_width_bucket_y)

### Fill Na values with latitude and longitude columns


In [ ]:
df_prediction['median_x'] = df_prediction['median_x'].fillna(df_prediction['longitude'])
df_prediction['median_y'] = df_prediction['median_y'].fillna(df_prediction['latitude'])

In [ ]:
df_prediction[['id','median_x','median_y']].drop_duplicates(inplace=True)

,id,median_x,median_y
0,1,-43.323649,-22.882424
2,2,-43.235302,-22.899506
4,3,-43.293874,-22.882424
6,4,-43.233350,-22.908496
8,5,-43.304125,-22.886470
...,...,...,...
310866,156638,-43.247993,-22.866691
310868,156639,-43.268981,-22.841968
310870,156640,-43.191373,-22.983116
310872,156641,-43.270446,-22.841518


## Compare the distance between predicted and real values

In [ ]:
answer = pd.read_json(answer_files[0], encoding='latin-1')
answer['latitude'] = answer['latitude'].str.replace(',', '.').astype(float)
answer['longitude'] = answer['longitude'].str.replace(',', '.').astype(float)
answer

,id,latitude,longitude
0,1,-22.88206,-43.32547
1,2,-22.90037,-43.23099
2,3,-22.88279,-43.29586
3,4,-22.90811,-43.23055
4,5,-22.88593,-43.30216
...,...,...,...
156637,156638,-22.87820,-43.24119
156638,156639,-22.83674,-43.28420
156639,156640,-22.96980,-43.18537
156640,156641,-22.84024,-43.27761


In [ ]:
# Function to calculate the distance between two points in a sphere in meters
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radius of Earth in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    meters = R * c
    return meters

final_df = answer.merge(df_prediction, on='id')

final_df['error_meters'] = df.apply(
    lambda row: haversine(row['latitude'], row['longitude'], row['median_y'], row['median_x']), axis=1
)



## Statistics

In [ ]:
mean_error = final_df['error_meters'].mean()
std_deviation = final_df['error_meters'].std()
median_error = final_df['error_meters'].median()

print(f'Average error in meters: {mean_error:.2f} m')
print(f'Standard deviation of the error in meters: {std_deviation:.2f} m')
print(f'Median error in meters: {median_error:.2f} m')

Média do erro em metros: 1384.85 m
Desvio padrão do erro em metros: 2259.66 m
Mediana do erro em metros: 645.63 m


## The baseline model, predict the last position of the bus as the next position

In [ ]:
# Get only odd rows
baseline = join_df[1::2]

baseline = baseline[['id','longitude','latitude']].drop_duplicates()

In [ ]:
baseline_df = answer.merge(baseline, on='id', how='inner', suffixes=('_real', '_predicted'))

baseline_df['error_meters'] = baseline_df.apply(
    lambda row: haversine(row['latitude_real'], row['longitude_real'], row['latitude_predicted'], row['longitude_predicted']), axis=1
)

# Statistics of the error in meters
mean_error = baseline_df['error_meters'].mean()
std_deviation = baseline_df['error_meters'].std()
median_error = baseline_df['error_meters'].median()

print(f'Média do erro em metros: {mean_error:.2f} m')
print(f'Desvio padrão do erro em metros: {std_deviation:.2f} m')
print(f'Mediana do erro em metros: {median_error:.2f} m')

Média do erro em metros: 5326.49 m
Desvio padrão do erro em metros: 4803.44 m
Mediana do erro em metros: 4180.00 m
